In [ ]:
import csv
import re
import shutil
from tempfile import NamedTemporaryFile

# Filepaths
output_file = 'replacement_sample_data.csv'  # The file where the modified data will be stored 
input_file = 'sample_data.csv'               # The file containing the original data
replacement_file = 'Replacement_words.csv'   # The file containing original and replacement words

# Create a temporary file to write the changes before moving it to the final location
tempfile = NamedTemporaryFile(mode='w', delete=False, newline='', encoding='utf-8')

def load_replacements(replacement_file):
    """
    Load word replacements from a CSV file into a dictionary.
    The right column contains words to be replaced by the corresponding words in the left column.
    """
    replacements = {}
    with open(replacement_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            original, replacement = row[1].lower(), row[0].lower()  # Ensure lowercase comparison
            replacements[original] = replacement
    return replacements

def replace_words_in_text(text, replacements):
    """
    Replace words or phrases (including bi-grams and tri-grams) in the input text according to the replacements dictionary.
    """
    # Sort replacements by length (so tri-grams are checked before bi-grams, etc.)
    sorted_replacements = sorted(replacements.keys(), key=lambda x: len(x.split()), reverse=True)
    
    # Create a pattern to match any n-grams (bi-grams, tri-grams, etc.)
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in sorted_replacements) + r')\b', re.IGNORECASE)
    
    # Function to preserve the original case of the replacement
    def replacement_func(match):
        original_word = match.group(0).lower()
        return replacements.get(original_word, original_word)
    
    # Replace matched phrases in the text using the generated pattern
    return pattern.sub(replacement_func, text)

def process_file(input_file, output_file, replacements):
    """
    Read the input file, perform word replacements, and write the modified content to the output file.
    """
    # Open input and temporary output files
    with open(input_file, 'r', encoding='utf-8') as infile, tempfile:
        reader = csv.reader(infile, delimiter=',', quotechar='"')
        writer = csv.writer(tempfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        # Write the header if present
        header = next(reader)
        writer.writerow(header)
        
        for row in reader:
            new_row = [replace_words_in_text(cell, replacements) for cell in row]
            writer.writerow(new_row)

    # Replace the original file with the modified file
    shutil.move(tempfile.name, output_file)

def main():
    # Load the replacement words from the replacement CSV file
    replacements = load_replacements(replacement_file)

    # Process the input file and apply the replacements
    process_file(input_file, output_file, replacements)

if __name__ == "__main__":
    main()
